In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np

In [0]:
def format_file(file_name, path="."): #path="drive/My Drive/JSPLIB/"
    jobs = []
    with open(path+"/instances/"+file_name,"r") as file:    
        f = file.readlines()
        for i in range(5, len(f)):
            jobs.append(f[i].split())
            
        return np.array(jobs), f[4]

In [0]:
class Task:
    def __init__(self, resource, duration, job, number):
        self.number = number
        self.resource = resource
        self.duration = duration
        self.job = job

In [0]:
class Job:
    def __init__(self, number):
        self.tasks = []
        self.number = number
        self.next_availability = 0
        self.current_task_idx = -1
        self.is_done = False
        
    def add_task(self, task):
        self.tasks.append(task)
        
    def set_next_availability(self, new_duration):
        self.next_availability = new_duration
        if (self.current_task_idx == len(self.tasks) - 1): # if we've already reached the last task
            self.is_done = True
            
    def get_next_task(self, cycle):
        if ((self.next_availability <= cycle) & (not self.is_done)):          
            self.current_task_idx += 1
            return self.tasks[self.current_task_idx]
        return None

In [0]:
class Resource:
    def __init__(self, number):
        self.number = number
        self.next_availability = 0
        self.current_task = None
        self.tmp_tasks = []
        self.all_tasks = []
        
    def set_next_availability(self, new_duration):
        self.next_availability = new_duration

In [0]:
def insert_task(task, priority): 
    '''Insert task in its corresponding ressource waiting list with respect to %s priority'''%(priority)
    sorted_list = task.resource.tmp_tasks
    index=len(sorted_list) # Insert in last position if no condition is satisfied
    # Searching for the position 
    for i in range(len(sorted_list)): 
        if (((sorted_list[i].duration < task.duration) & (priority == 'LPT')) | ((sorted_list[i].duration > task.duration) & (priority == 'SPT'))):             
            index = i 
            break
      
    # Inserting in the list
    sorted_list.insert(index, task)

In [0]:
def insert_task_randomly(task):
    '''Insert task in its corresponding ressource waiting list randomly'''
    random_index = round(np.random.random_sample() * len(task.resource.tmp_tasks))

    # Inserting in the list
    task.resource.tmp_tasks.insert(random_index, task)

In [0]:
def reorder_tasks(resources, waiting_tasks, priority):
    for task in waiting_tasks:
        if priority:
          insert_task(task, priority)
        else:
          insert_task_randomly(task)

    print("Resources")    
    for resource in resources:
        for tmp_task in resource.tmp_tasks: 
            old_av = max(resource.next_availability, tmp_task.job.next_availability)
            resource.set_next_availability(old_av + tmp_task.duration) # Update availability on resource
            tmp_task.job.set_next_availability(old_av + tmp_task.duration) # Update availability on job
            resource.all_tasks.append(("J{} - T{}".format(int(tmp_task.job.number), int(tmp_task.number)), old_av ,resource.next_availability))

        print("R%i"%(resource.number))        
        print(resource.all_tasks)
        # Flash resources waiting tasks
        resource.tmp_tasks = []
    
    print("-"*100) # Display separator

In [0]:
def get_available_tasks(jobs, cycle):
    tasks = []
    for job in jobs:
        task = job.get_next_task(cycle)
        if (task):
            tasks.append(task)
        else:
            if(job.is_done):
                jobs.remove(job)
    print("Available Tasks (Not assigned to a resource)")
    print([(task.job.number, task.resource.number) for task in tasks])
    return tasks, jobs

In [0]:
# Main Program
jobs = []
resources = []
cycle = 0
priority = None # "LPT", "SPT", None(Random)
job_list, shape = format_file("ft06")
number_of_ressources = int(shape.split()[1])

# Initialization
## Create Ressource instances
for i in range(number_of_ressources):
    resources.append(Resource(number=i))
    
## Create Job then Task instances    
job_index = 1
for line in job_list:
    job = Job(job_index)
    for k in range(0,len(line),2):
        resource_nbr = int(line[k])
        duration = int(line[k+1])
        task = Task(resources[resource_nbr], duration, job, k/2)
        job.add_task(task)
    
    jobs.append(job)
    job_index+=1

In [0]:
# Launching the program
print("Jobshop 6x6 with an %s priority"%(priority))
print("-"*100)
while(jobs != []):
    print("Cycle: {}".format(cycle))    
    waiting_tasks, new_jobs = get_available_tasks(jobs, cycle) # Tasks that needs to be assigned to one ressource
    reorder_tasks(resources, waiting_tasks, priority)
    jobs = new_jobs # Update job list: if a job is done, it will be removed from the list
    cycle += 1

Jobshop 6x6 with an None priority
----------------------------------------------------------------------------------------------------
Cycle: 0
Available Tasks (Not assigned to a resource)
[(1, 2), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1)]
Resources
R0
[]
R1
[('J4 - T0', 0, 5), ('J2 - T0', 5, 13), ('J6 - T0', 13, 16)]
R2
[('J3 - T0', 0, 5), ('J5 - T0', 5, 14), ('J1 - T0', 14, 15)]
R3
[]
R4
[]
R5
[]
----------------------------------------------------------------------------------------------------
Cycle: 1
Available Tasks (Not assigned to a resource)
[]
Resources
R0
[]
R1
[('J4 - T0', 0, 5), ('J2 - T0', 5, 13), ('J6 - T0', 13, 16)]
R2
[('J3 - T0', 0, 5), ('J5 - T0', 5, 14), ('J1 - T0', 14, 15)]
R3
[]
R4
[]
R5
[]
----------------------------------------------------------------------------------------------------
Cycle: 2
Available Tasks (Not assigned to a resource)
[]
Resources
R0
[]
R1
[('J4 - T0', 0, 5), ('J2 - T0', 5, 13), ('J6 - T0', 13, 16)]
R2
[('J3 - T0', 0, 5), ('J5 - T0', 5, 14),